# **HOMEWORK 1 - Regressione Lineare**

In questo homework dovrete:

1. Scrivere una funzione di pipeline che deve gestire l' allenamento di un modello di regressione lineare al variare degli iperparametri forniti. Nello specifico:
    * Deve applicare la PCA, se presente.
    
    * Deve applicare la standardizzazione, se presente.

    * Deve applicare la regolarizzazione, se presente.

    * Deve allenare il modello di regressione lineare.

    * Deve calcolare la MAE.

2. Scrivere una funzione che utilizzi la `pipeline` definita al punto 1 e che testi tutte le configurazioni possibili presenti in `configs`. Nel dettaglio la funzione deve:
    * Dividere il dataset in train e validation.

    * Calcolare, grazie alla funzione `pipeline` definita al punto 1, quale configurazione ottiene il punteggio migliore (quale configurazione ha la MAE di validation più bassa).

3. Scrivere una funzione che utilizzi la configurazione migliore prodotta dalla funzione definita al punto 2 e la testi sul test set. 

4. Stampare:
    * La migliore configurazione

    * Il miglior MAE di validation 

    * Il migliore MAE di train

    * Il MAE di test 


Il codice che di seguito trovate già fornito deve essere utilizzato per la risoluzione dell' homework, **NON MODIFICATELO IN ALCUN MODO**.

## **Dataset Wine Quality White**

Il dataset da utilizzare è `wine-quality-white` della libreria `scikit-learn`. Il dataset contiene 11 variabili numeriche + 1 di target che classifica il vino in diverse categorie di qualità. Per il nostro obiettivo la variabile di target è considerata come `float`, permettendoci di applicare la regressione lineare. All' interno del dataset sono contenuti 4898 campioni. 

In [2]:
# Questa cella contiene tutte le librerie di cui necessitate per risolvere l' homework.
# Ricordate di eseguirla prima di iniziare.

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA
from sklearn.datasets import fetch_openml
from sklearn.utils import shuffle
from sklearn.preprocessing import StandardScaler

In [7]:
hyperparams = {
    # PCA
    'use_pca': [True, False],
    'pca_standardize': [True, False],
    'pca_components': [3, 5, 10],
    # Data standardization
    'data_standardize': [True, False],
    # Regularization l2
    'use_regularization': [True, False],
    'reg_lambda': [0.1, 1, 10],
}

# Calcoliamo tutte le possibili combinazioni di iperparametri
import itertools
combinations = list(itertools.product(*hyperparams.values()))
configs = [dict(zip(hyperparams.keys(), combination)) for combination in combinations]

# Evitiamo le combinazioni non valide
for config in configs:
    if not config['use_pca']:
        config['pca_standardize'] = None
        config['pca_components'] = None
    if not config['use_regularization']:
        config['reg_lambda'] = None
configs = set([tuple(config.items()) for config in configs])

# Convertiamo di nuovo in lista di dizionari
configs = [dict(config) for config in configs]
print(f'Numero di combinazioni: {len(configs)}')

Numero di combinazioni: 56
4898


In `configs` avete una lista di dizionari, ogni dizionario contiene una possibile combinazione di hyperparametri da utilizzare nella fase di training. 

In [6]:
# Carica il dataset Wine Quality White
data = fetch_openml(name='wine-quality-white', version=1, as_frame=True)
X = data.data
y = data.target.astype(float)  # Assicura che il target sia float per la regressione

def pipeline(X_train, y_train, X_val, y_val, hyperparams):
    """
    Addestra un modello di regressione lineare con eventuale PCA e regolarizzazione L2.
    """
    if hyperparams['use_pca']:
        n_components=hyperparams.get("pca_components")
        if hyperparams['pca_standardize']:
            SC = StandardScaler() 
            X_train=SC.fit_transform(X_train)
            X_val=SC.transform(X_val)

        pca= PCA(n_components)
        X_train=pca.fit_transform(X_train)
        X_val=pca.transform(X_val)
    if hyperparams['data_standardize']:
        SC = StandardScaler()
        X_train=SC.fit_transform(X_train)
        X_val=SC.transform(X_val)

    # Aggiunge il termine costante ai dati
    ones_train=np.ones(X_train.shape[0])
    ones_val=np.ones(X_val.shape[0])
    X_train=np.c_[X_train, ones_train]
    X_val=np.c_[X_val, ones_val]

    # Calcolo della soluzione di regressione lineare
    if hyperparams['use_regularization']:
        Xt_X=X_train.T@X_train
        alfa=hyperparams.get("reg_lambda")
        I=np.identity(Xt_X.shape[0])
        inversa=np.linalg.inv(Xt_X+alfa*I)
        Xt_Y=X_train.T@y_train
        parametri=inversa@Xt_Y
    else:
        inversa=np.linalg.inv(X_train.T@X_train)
        Xt_Y=X_train.T@y_train
        parametri=inversa@Xt_Y

    # Calcolo predizioni
    valori_predetti_train= X_train @ parametri

    valori_predetti_val=X_val@parametri

    # Calcola il MAE
    valori_assoluti_train= np.abs(y_train-valori_predetti_train)
    MAE_train=np.mean(valori_assoluti_train)

    valori_assoluti_val=np.abs(y_val-valori_predetti_val)
    MAE_val=np.mean(valori_assoluti_val)
    return MAE_train, MAE_val

# Dividi il dataset in training e test set

train_porzione=0.8

lunghezza_train=int(len(X)*train_porzione)

X_train=X[:lunghezza_train]
y_train=y[:lunghezza_train]

X_test=X[lunghezza_train:]
y_test=y[lunghezza_train:]
# Dividi il training set in training set effettivo e validation set

train_effettivo_porzione=0.75 
#0.75 del 0.8 della lunghezza originale (e cioè 0.6 * lunghezza originale), di modo che il dataset sia diviso in 60-20-20

lunghezza_train_effettivo=int(len(X)*train_effettivo_porzione)
X_train_effettivo=X_train[:lunghezza_train_effettivo]
y_train_effettivo=y_train[:lunghezza_train_effettivo]

X_val=X_train[lunghezza_train_effettivo:]
y_val=y_train[lunghezza_train_effettivo:]


# Trova la configurazione di iperparametri migliore
def configurazione_migliore(X_train, y_train, X_val, y_val, configs):
    MAE_val_migliore=float('inf')
    for hyperparams in configs:
        X_train_copy=X_train.copy()
        X_val_copy=X_val.copy()
        MAE_train, MAE_val=pipeline(X_train_copy, y_train, X_val_copy, y_val, hyperparams)
        if(MAE_val<MAE_val_migliore):
            MAE_val_migliore=MAE_val
            iperparametro_migliore=hyperparams
    return iperparametro_migliore, MAE_val_migliore

miglior_hyperparams, miglior_MAE_val= configurazione_migliore(X_train_effettivo, y_train_effettivo, X_val, y_val, configs)

# Riallena il modello sul training set completo
MAE_train_migliore, MAE_test= pipeline(X_train, y_train, X_test, y_test, miglior_hyperparams)

# Calcola il MAE sul test set
        #ho usato precedentemente la pipeline per calcolarmi anche il MAE sul test set...

# Stampa  risultati
print(f'Miglior configurazione: {miglior_hyperparams}')
print(f'Miglior MAE di validation: {miglior_MAE_val}')
print(f'Miglior MAE di train: {MAE_train_migliore}')
print(f'MAE sul test set: {MAE_test}')


/usr/local/lib/python3.11/dist-packages/sklearn/datasets/_openml.py:968: FutureWarning: The default value of `parser` will change from `'liac-arff'` to `'auto'` in 1.4. You can set `parser='auto'` to silence this warning. Therefore, an `ImportError` will be raised from 1.4 if the dataset is dense and pandas is not installed. Note that the pandas parser may return different data types. See the Notes Section in fetch_openml's API doc for details.
  warn(


Miglior configurazione: {'use_pca': False, 'pca_standardize': None, 'pca_components': None, 'data_standardize': True, 'use_regularization': True, 'reg_lambda': 10}
Miglior MAE di validation: 0.5918053837748496
Miglior MAE di train: 0.5942579843664354
MAE sul test set: 0.5483824028925265
